**Вопросы для повторения:**
* что такое `std::mutex`? зачем он нужен? что внутри?
* что такое `std::lock_guard`? чем не устраивают `std::mutex::lock`, `std::mutex::unlock`?
* предположим, у нас есть многопоточная очередь задач `MTTasksQueue`. В чём здесь проблема? Как её будем исправлять?

```c++
void run_queued_task(MTTasksQueue& q)
{
    if (!q.empty())
        run_task(q.pop());
}
```

* а в чём может быть проблема с той же самой очередью тут? Какие есть варианты её исправить?

```c++
class MTTasksQueue
{
public:
    using Task = std::function<void(void)>;
    
    ...

    void pop_and_run()
    {
        std::lock_guard guard(mtx);

        if (!tasks_queue.empty())
        {        
            const auto task = std::move(tasks_queue.back());
            tasks_queue.pop_back();
            task();
        }        
    }

private:
    std::mutex mtx;
    std::queue<Task> tasks_queue;
};
```

<details>
<summary>ответ</summary>
<p>recursive_mutex и реорганизация кода. Что такое и как устроен recursive_mutex? Когда его следует использовать? Почему в данном случае recursive_mutex - плохое решение? Если остаться на обычном mutex, как следует с ним поступить?</p>
</details>

* Что такое deadlock? Каким минимальным числом потоков и mutex-ов устроить deadlock?

<details>
<summary>замечание</summary>
<p>"на самом деле", дважды вызов lock у std::mutex на одном потоке - это не обязательно deadlock. Документация утверждает, что это UB, и _скорее всего_ вы получите deadlock, но некоторые реализации могут задетектить ситуацию и бросить exception, но в общем случае всё совсем плохо - это UB</p>
<p><a href="https://en.cppreference.com/w/cpp/thread/mutex/lock">proof</a></p>
</details>


* что такое и зачем нужны `shared_mutex`, `unique_lock`, `shared_lock` ?
* какие есть гарантии потокобезопасности на `std::shared_ptr` ?